In [7]:
import nltk
import sklearn_crfsuite

from copy import deepcopy
from collections import defaultdict

from sklearn_crfsuite.metrics import flat_classification_report

from ner_evaluation.ner_eval import collect_named_entities
from ner_evaluation.ner_eval import compute_metrics

## Train a CRF on the CoNLL 2002 NER Spanish data

In [2]:
nltk.corpus.conll2002.fileids()
train_sents = list(nltk.corpus.conll2002.iob_sents('esp.train'))
test_sents = list(nltk.corpus.conll2002.iob_sents('esp.testb'))

In [3]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]


def sent2labels(sent):
    return [label for token, postag, label in sent]


def sent2tokens(sent):
    return [token for token, postag, label in sent]

## Feature Extraction

In [4]:
%%time
X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

X_test = [sent2features(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]

CPU times: user 1.24 s, sys: 115 ms, total: 1.35 s
Wall time: 1.35 s


## Training

In [5]:
%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

CPU times: user 33.2 s, sys: 188 ms, total: 33.4 s
Wall time: 33.4 s


## Performance per label type per token

In [8]:
y_pred = crf.predict(X_test)
labels = list(crf.classes_)
labels.remove('O') # remove 'O' label from evaluation
sorted_labels = sorted(labels,key=lambda name: (name[1:], name[0])) # group B and I results
print(flat_classification_report(y_test, y_pred, labels=sorted_labels, digits=3))

              precision    recall  f1-score   support

       B-LOC      0.810     0.784     0.797      1084
       I-LOC      0.690     0.637     0.662       325
      B-MISC      0.731     0.569     0.640       339
      I-MISC      0.699     0.589     0.639       557
       B-ORG      0.807     0.832     0.820      1400
       I-ORG      0.852     0.786     0.818      1104
       B-PER      0.850     0.884     0.867       735
       I-PER      0.893     0.943     0.917       634

   micro avg      0.813     0.787     0.799      6178
   macro avg      0.791     0.753     0.770      6178
weighted avg      0.809     0.787     0.796      6178



## Performance over full named-entity

In [9]:
test_sents_labels = []
for sentence in test_sents:
    sentence = [token[2] for token in sentence]
    test_sents_labels.append(sentence)

In [10]:
index = 2
true = collect_named_entities(test_sents_labels[index])
pred = collect_named_entities(y_pred[index])

In [11]:
true

[Entity(e_type='MISC', start_offset=12, end_offset=12),
 Entity(e_type='LOC', start_offset=15, end_offset=15),
 Entity(e_type='PER', start_offset=37, end_offset=39),
 Entity(e_type='ORG', start_offset=45, end_offset=46)]

In [12]:
pred

[Entity(e_type='MISC', start_offset=12, end_offset=12),
 Entity(e_type='LOC', start_offset=15, end_offset=15),
 Entity(e_type='PER', start_offset=37, end_offset=39),
 Entity(e_type='LOC', start_offset=45, end_offset=46)]

In [13]:
index = 2
true = collect_named_entities(test_sents_labels[index])
pred = collect_named_entities(y_pred[index])

In [14]:
true

[Entity(e_type='MISC', start_offset=12, end_offset=12),
 Entity(e_type='LOC', start_offset=15, end_offset=15),
 Entity(e_type='PER', start_offset=37, end_offset=39),
 Entity(e_type='ORG', start_offset=45, end_offset=46)]

In [15]:
pred

[Entity(e_type='MISC', start_offset=12, end_offset=12),
 Entity(e_type='LOC', start_offset=15, end_offset=15),
 Entity(e_type='PER', start_offset=37, end_offset=39),
 Entity(e_type='LOC', start_offset=45, end_offset=46)]

In [16]:
compute_metrics(true, pred)

({'strict': {'correct': 3,
   'incorrect': 1,
   'partial': 0,
   'missed': 0,
   'spurius': 0,
   'possible': 4,
   'actual': 4,
   'precision': 0.75,
   'recall': 0.75},
  'ent_type': {'correct': 3,
   'incorrect': 1,
   'partial': 0,
   'missed': 0,
   'spurius': 0,
   'possible': 4,
   'actual': 4,
   'precision': 0.75,
   'recall': 0.75},
  'partial': {'correct': 4,
   'incorrect': 0,
   'partial': 0,
   'missed': 0,
   'spurius': 0,
   'possible': 4,
   'actual': 4,
   'precision': 1.0,
   'recall': 1.0},
  'exact': {'correct': 4,
   'incorrect': 0,
   'partial': 0,
   'missed': 0,
   'spurius': 0,
   'possible': 4,
   'actual': 4,
   'precision': 1.0,
   'recall': 1.0}},
 {'MISC': {'strict': {'correct': 1,
    'incorrect': 0,
    'partial': 0,
    'missed': 0,
    'spurius': 0},
   'ent_type': {'correct': 1,
    'incorrect': 0,
    'partial': 0,
    'missed': 0,
    'spurius': 0},
   'partial': {'correct': 0,
    'incorrect': 0,
    'partial': 0,
    'missed': 0,
    'spurius': 

In [17]:
to_test = [2,4,12,14]

In [18]:
index = 2
true_named_entities_type = defaultdict(list)
pred_named_entities_type = defaultdict(list)

for true in collect_named_entities(test_sents_labels[index]):
    true_named_entities_type[true.e_type].append(true)

for pred in collect_named_entities(y_pred[index]):
    pred_named_entities_type[pred.e_type].append(pred)

In [19]:
true_named_entities_type

defaultdict(list,
            {'MISC': [Entity(e_type='MISC', start_offset=12, end_offset=12)],
             'LOC': [Entity(e_type='LOC', start_offset=15, end_offset=15)],
             'PER': [Entity(e_type='PER', start_offset=37, end_offset=39)],
             'ORG': [Entity(e_type='ORG', start_offset=45, end_offset=46)]})

In [20]:
pred_named_entities_type

defaultdict(list,
            {'MISC': [Entity(e_type='MISC', start_offset=12, end_offset=12)],
             'LOC': [Entity(e_type='LOC', start_offset=15, end_offset=15),
              Entity(e_type='LOC', start_offset=45, end_offset=46)],
             'PER': [Entity(e_type='PER', start_offset=37, end_offset=39)]})

In [21]:
true_named_entities_type['LOC']

[Entity(e_type='LOC', start_offset=15, end_offset=15)]

In [22]:
pred_named_entities_type['LOC']

[Entity(e_type='LOC', start_offset=15, end_offset=15),
 Entity(e_type='LOC', start_offset=45, end_offset=46)]

In [23]:
compute_metrics(true_named_entities_type['LOC'], pred_named_entities_type['LOC'])

({'strict': {'correct': 1,
   'incorrect': 0,
   'partial': 0,
   'missed': 0,
   'spurius': 1,
   'possible': 1,
   'actual': 2,
   'precision': 0.5,
   'recall': 1.0},
  'ent_type': {'correct': 1,
   'incorrect': 0,
   'partial': 0,
   'missed': 0,
   'spurius': 1,
   'possible': 1,
   'actual': 2,
   'precision': 0.5,
   'recall': 1.0},
  'partial': {'correct': 1,
   'incorrect': 0,
   'partial': 0,
   'missed': 0,
   'spurius': 1,
   'possible': 1,
   'actual': 2,
   'precision': 0.5,
   'recall': 1.0},
  'exact': {'correct': 1,
   'incorrect': 0,
   'partial': 0,
   'missed': 0,
   'spurius': 1,
   'possible': 1,
   'actual': 2,
   'precision': 0.5,
   'recall': 1.0}},
 {'MISC': {'strict': {'correct': 0,
    'incorrect': 0,
    'partial': 0,
    'missed': 0,
    'spurius': 0},
   'ent_type': {'correct': 0,
    'incorrect': 0,
    'partial': 0,
    'missed': 0,
    'spurius': 0},
   'partial': {'correct': 0,
    'incorrect': 0,
    'partial': 0,
    'missed': 0,
    'spurius': 0},


## results over all messages

In [24]:
metrics_results = {'correct': 0, 'incorrect': 0, 'partial': 0,
                   'missed': 0, 'spurius': 0, 'possible': 0, 'actual': 0}

# overall results
results = {'strict': deepcopy(metrics_results),
           'ent_type': deepcopy(metrics_results),
           'partial':deepcopy(metrics_results),
           'exact':deepcopy(metrics_results)
          }

# results aggregated by entity type
evaluation_agg_entities_type = {e: deepcopy(results) for e in ['LOC','PER','ORG','MISC']}

for true_ents, pred_ents in zip(test_sents_labels, y_pred):
    
    # compute results for one message
    tmp_results, tmp_agg_results = compute_metrics(collect_named_entities(true_ents),collect_named_entities(pred_ents))

    # aggregate overall results
    for eval_schema in results.keys():
        for metric in metrics_results.keys():
            results[eval_schema][metric] += tmp_results[eval_schema][metric]

    # aggregate results by entity type
    for e_type in ['LOC','PER','ORG','MISC']:
        for eval_schema in tmp_agg_results[e_type]:
            for metric in tmp_agg_results[e_type][eval_schema]:
                evaluation_agg_entities_type[e_type][eval_schema][metric] += tmp_agg_results[e_type][eval_schema][metric]

In [25]:
results

{'strict': {'correct': 2783,
  'incorrect': 600,
  'partial': 0,
  'missed': 176,
  'spurius': 139,
  'possible': 3559,
  'actual': 3522},
 'ent_type': {'correct': 2860,
  'incorrect': 523,
  'partial': 0,
  'missed': 176,
  'spurius': 139,
  'possible': 3559,
  'actual': 3522},
 'partial': {'correct': 3278,
  'incorrect': 0,
  'partial': 105,
  'missed': 176,
  'spurius': 139,
  'possible': 3559,
  'actual': 3522},
 'exact': {'correct': 3278,
  'incorrect': 105,
  'partial': 0,
  'missed': 176,
  'spurius': 139,
  'possible': 3559,
  'actual': 3522}}

In [26]:
evaluation_agg_entities_type

{'LOC': {'strict': {'correct': 844,
   'incorrect': 168,
   'partial': 0,
   'missed': 57,
   'spurius': 38,
   'possible': 0,
   'actual': 0},
  'ent_type': {'correct': 855,
   'incorrect': 157,
   'partial': 0,
   'missed': 49,
   'spurius': 32,
   'possible': 0,
   'actual': 0},
  'partial': {'correct': 0,
   'incorrect': 0,
   'partial': 0,
   'missed': 49,
   'spurius': 0,
   'possible': 0,
   'actual': 0},
  'exact': {'correct': 0,
   'incorrect': 0,
   'partial': 0,
   'missed': 49,
   'spurius': 0,
   'possible': 0,
   'actual': 0}},
 'PER': {'strict': {'correct': 646,
   'incorrect': 104,
   'partial': 0,
   'missed': 19,
   'spurius': 15,
   'possible': 0,
   'actual': 0},
  'ent_type': {'correct': 651,
   'incorrect': 99,
   'partial': 0,
   'missed': 17,
   'spurius': 13,
   'possible': 0,
   'actual': 0},
  'partial': {'correct': 0,
   'incorrect': 0,
   'partial': 0,
   'missed': 17,
   'spurius': 0,
   'possible': 0,
   'actual': 0},
  'exact': {'correct': 0,
   'incorre